In [177]:
import re
import numpy as np
import pandas as pd
import math
import sys
import string
import json

import sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
from datasets import load_dataset

from konlpy.tag import Okt
okt = Okt()
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

BM25

In [27]:
class BM25:
    def __init__(self, k1=1.5, b=0.75): #k1, b 는 hyperparameter 
        self.k1= k1 
        self.b = b
    def fit(self, corpus):
        tf = [] # 단어 빈도
        df = {} # 문서 빈도
        idf = {} # 역문서 빈도
        
        corpus_size = 0 # 문서 수
        doc_len = [] # 각 문서의 길이 저장
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))

            #문서마다 tf 구하기
            frequencies = {}
            for term in document:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count
            tf.append(frequencies)

            # 문서들의 단어 빈도 구하기
            for term, _ in frequencies.items():
                df_count = df.get(term, 0)+ 1
                df[term] = df_count
        
        for term, freq in df.items(): # freq는 해당 단어를 포함하는 문서 개수
            idf[term] = math.log(1+ (corpus_size - freq + 0.5)) / (freq + 0.5) #IDF
        
        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len = sum(doc_len) / corpus_size #문서들의 평균 길이
        return self
    
    def score_cal(self, query, index):
        score = 0.0
        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term not in frequencies:
                continue
            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 +1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len)
            score += (numerator / denominator) 
        return score
        
    def search(self, query):
        scores = [self.score_cal(query, index) for index in range(self.corpus_size_)]
        return scores

In [202]:
datasets = load_dataset("squad_kor_v1")
question_list = [data['question'] for data in datasets['validation']]
context_list = [data['context'] for data in datasets['validation']]
dataset = pd.DataFrame({'question':question_list, 'context':context_list})
dataset.head()

,question,context
0,임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...
1,1989년 6월 30일 평양축전에 대표로 파견 된 인물은?,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...
2,임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...
3,임종석을 검거한 장소는 경희대 내 어디인가?,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...
4,임종석이 조사를 받은 뒤 인계된 곳은 어딘가?,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...


In [203]:
#동일한 문서가 많으니 하나씩만 골라내기위한 인덱스 찾기
_, index = np.unique(np.array(dataset['context']), return_index = True)
dataset = dataset.iloc[index]
dataset.reset_index(drop=True, inplace=True)
dataset['y'] = range(0, dataset.shape[0] , 1)

In [204]:
dataset

,question,context,y
0,일본의 독도 영유권 주장 당시 김영삼이 당대 외교활동을 강력하게 비꼬아 표현한 상대...,"""나는 대통령 때 외무부에 지시해 독도 인근 해역에 배를 엄청나게 띄워 해상시위를 ...",0
1,정부의 헌법개정안 준비 과정에 대해서 청와대 비서실이 아니라 국무회의 중심으로 이뤄...,"""내각과 장관들이 소외되고 대통령비서실의 권한이 너무 크다"", ""행보가 비서 본연의...",1
2,바이오쇼크에서 플레이어가 조심해야 하는 캐릭터는 무엇인가?,"""빅 대디""는 플레이어가 주의해야 하는 캐릭터로, 장갑한 잠수복을 입었으며 유전적으...",2
3,서양 철학이라는 용어의 지정학적 경계는 몇세기에 걸쳐 형성 되었는가?,"""서양 철학""이라는 용어의 지정학적 경계는 19-20세기에 걸쳐 형성되었다. 이 시...",3
4,고위급 탈북자는 천안함 침몰 사건이 북한이 몇년동안 철저히 준비해온 사건이라고 주장...,'북한의 군사문제와 군수산업에 정통한 한 고위급 탈북자'는 이 사건이 10년 동안 ...,4
...,...,...,...
955,1920년대 전후반에 일본은 무엇의 여파로 불황 상태에 빠졌나?,"히로히토가 천황 자리에 오르던 1920년대 전후반, 일본은 세계 대공황의 여파로 극...",955
956,가와무라 사후에 히로히토 형제의 양육을 맡았던 사람은?,히로히토는 가와무라가 세상을 떠난 1904년 11월까지 가와무라의 저택에서 지냈으며...,956
957,히로히토에게 박물과 물리를 가르쳤던 교수는?,"히로히토는 어렸을 때부터 자연에 관심을 보였다. 히로히토가 중등과에 다닐 때, 박물...",957
958,아몬의 일이 적혀있는 성경의 이름은 무엇인가?,"히브리어 성경에 기록된 바에 따르면, 아몬은 아버지 므낫세의 정책을 답습해 우상숭배...",958


In [205]:
def preprocessing(pre_docs):
    for i in range(len(pre_docs)):
        pre_docs[i] = re.sub('\《.*\》|\s-\s.*', '', pre_docs[i])
        pre_docs[i] = re.sub('\(.*\)|\s-\s.*', '', pre_docs[i])
        #필드의 태그를 모두 제거
        pre_docs[i] = re.sub('(<([^>]+)>)', '', pre_docs[i])
        # 개행문자 제거
        pre_docs[i] = re.sub('\\\\n', ' ', pre_docs[i])
        #한글 숫자만 제외 모두 제외
        pre_docs[i] = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ0-9]', ' ', pre_docs[i])
        pre_docs[i] = ' '.join(pre_docs[i].split())
    return pre_docs

In [206]:
def mecab_pos(doc_li):
    toks_docs =[]
    for doc in doc_li:
        doc_pos = []
        for word in mecab.pos(doc):
            if word[1] in ['NNG', 'NNP','NP','NNB','NR']: #일반명사, 고유명사, 대명사, 의존명사, 수사
                doc_pos.append(word[0])
        toks_docs.append(doc_pos)
    return toks_docs

In [207]:
def mecab_morphs(doc_li):
    toks_docs = [mecab.morphs(row) for row in doc_li]
    return toks_docs

In [208]:
def okt_pos(doc_li):
    toks_docs = [okt.morphs(row) for row in doc_li]
    return toks_docs

In [209]:
def remove_stop_words(doc_li, q_tok=None):
    #stopwords 경로 설정
    stop_words_f = "./stopwords.txt"
    print(stop_words_f)
    with open(stop_words_f, "r", encoding='utf-8') as f:
            stop_words = f.readlines()
    stop_words = [stop_word.strip() for stop_word in stop_words]
    if not q_tok == None:
        
        q_tok_list = []
        for q_ in q_tok:
            q_result = [w for w in q_ if not w in stop_words]
            q_tok_list.append(q_result)

        doc_li_result = []
        for doc in doc_li:
            result = [w for w in doc if w not in stop_words]
            doc_li_result.append(result)
        return q_tok_list, doc_li_result 
    
    else:
        doc_li_result = []
        for doc in doc_li:
            result = [w for w in doc if w not in stop_words]
            doc_li_result.append(result)
        return doc_li_result
        

In [210]:
#####  mecab으로
#문서들 전처리
doc_list = preprocessing(dataset['context'].tolist()) #기본 전처리
doc_list = mecab_morphs(doc_list) #명사만
doc_list = remove_stop_words(doc_list) # 불용어처리

# 질문들 전처리
q_list = preprocessing(dataset['question'].tolist())
q_list = mecab_morphs(q_list)
q_list, _ = remove_stop_words(doc_list, q_list)

./stopwords.txt
./stopwords.txt


In [181]:
#####  okt으로 형태소
#문서들 전처리
doc_list = preprocessing(dataset['context'].tolist()) #기본 전처리
doc_list = okt_pos(doc_list) #명사만
doc_list = remove_stop_words(doc_list) # 불용어처리

# 질문들 전처리
q_list = preprocessing(dataset['question'].tolist())
q_list = okt_pos(q_list)
q_list, _ = remove_stop_words(doc_list, q_list)

./stopwords.txt
./stopwords.txt


In [211]:
# 문서들 fit
bm25 = BM25() 
bm25.fit(doc_list) #정제한 문서들

In [212]:
#bm25 질의에 대한 적합한 문서 예측
y_hat = []
for q_ in q_list:
    scores = bm25.search(q_)
    y_hat.append(scores.index(max(scores)))
#예측한 문서인덱스를 데이터프레임에 적재
dataset['bm25_y_hat'] = y_hat

In [213]:
dataset

,question,context,y,bm25_y_hat
0,일본의 독도 영유권 주장 당시 김영삼이 당대 외교활동을 강력하게 비꼬아 표현한 상대...,"""나는 대통령 때 외무부에 지시해 독도 인근 해역에 배를 엄청나게 띄워 해상시위를 ...",0,203
1,정부의 헌법개정안 준비 과정에 대해서 청와대 비서실이 아니라 국무회의 중심으로 이뤄...,"""내각과 장관들이 소외되고 대통령비서실의 권한이 너무 크다"", ""행보가 비서 본연의...",1,1
2,바이오쇼크에서 플레이어가 조심해야 하는 캐릭터는 무엇인가?,"""빅 대디""는 플레이어가 주의해야 하는 캐릭터로, 장갑한 잠수복을 입었으며 유전적으...",2,317
3,서양 철학이라는 용어의 지정학적 경계는 몇세기에 걸쳐 형성 되었는가?,"""서양 철학""이라는 용어의 지정학적 경계는 19-20세기에 걸쳐 형성되었다. 이 시...",3,3
4,고위급 탈북자는 천안함 침몰 사건이 북한이 몇년동안 철저히 준비해온 사건이라고 주장...,'북한의 군사문제와 군수산업에 정통한 한 고위급 탈북자'는 이 사건이 10년 동안 ...,4,4
...,...,...,...,...
955,1920년대 전후반에 일본은 무엇의 여파로 불황 상태에 빠졌나?,"히로히토가 천황 자리에 오르던 1920년대 전후반, 일본은 세계 대공황의 여파로 극...",955,955
956,가와무라 사후에 히로히토 형제의 양육을 맡았던 사람은?,히로히토는 가와무라가 세상을 떠난 1904년 11월까지 가와무라의 저택에서 지냈으며...,956,956
957,히로히토에게 박물과 물리를 가르쳤던 교수는?,"히로히토는 어렸을 때부터 자연에 관심을 보였다. 히로히토가 중등과에 다닐 때, 박물...",957,957
958,아몬의 일이 적혀있는 성경의 이름은 무엇인가?,"히브리어 성경에 기록된 바에 따르면, 아몬은 아버지 므낫세의 정책을 답습해 우상숭배...",958,958


TF-IDF

In [214]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
len(q_list), len(doc_list)

(960, 960)

In [215]:
# 사이킷런의 tf-idf default인자에 맞추기위해 재조정
q_list = [','.join(q_).replace(',',' ') for q_ in q_list]
doc_list = [','.join(doc).replace(',',' ') for doc in doc_list]

In [216]:
# q_list와 doc_list 합쳐 fit 한다.
# n-gram은 1~2로 한다.
q_list.extend(doc_list) # 질의와 문서를 학습하기위해 잠시 합쳐놓음
tfidf = TfidfVectorizer(ngram_range=(1,2)).fit(q_list)
len(tfidf.vocabulary_)

82763

In [217]:
tfidf.vocabulary_

{'일본': 58460,
 '영유권': 50354,
 '주장': 65378,
 '당시': 19045,
 '김영삼': 14087,
 '당대': 18943,
 '외교': 51689,
 '활동': 81203,
 '강력': 4993,
 '비꼬': 35741,
 '표현': 76462,
 '상대': 38528,
 '대통령': 20169,
 '일본 영유권': 58587,
 '영유권 주장': 50355,
 '주장 당시': 65418,
 '당시 김영삼': 19077,
 '김영삼 당대': 14110,
 '당대 외교': 18944,
 '외교 활동': 51709,
 '활동 강력': 81216,
 '강력 비꼬': 4997,
 '비꼬 표현': 35742,
 '표현 상대': 76474,
 '상대 대통령': 38547,
 '정부': 62795,
 '헌법': 79751,
 '개정안': 5608,
 '준비': 65699,
 '과정': 9404,
 '청와대': 69957,
 '비서실': 35974,
 '국무회의': 11040,
 '중심': 66019,
 '이뤄졌': 56176,
 '어야': 48137,
 '다고': 17691,
 '지적': 67165,
 '원로': 52733,
 '학자': 77578,
 '정부 헌법': 62985,
 '헌법 개정안': 79755,
 '개정안 준비': 5615,
 '준비 과정': 65702,
 '과정 청와대': 9465,
 '청와대 비서실': 69971,
 '비서실 국무회의': 35975,
 '국무회의 중심': 11041,
 '중심 이뤄졌': 66044,
 '이뤄졌 어야': 56181,
 '어야 다고': 48142,
 '다고 지적': 17801,
 '지적 원로': 67196,
 '원로 헌법': 52738,
 '헌법 학자': 79786,
 '바이오': 30071,
 '쇼크': 42049,
 '플레이어': 76852,
 '조심': 64423,
 '해야': 79002,
 '캐릭터': 72364,
 '인가': 57548,
 '바이오 쇼크': 30072,
 '쇼크 플레이어

In [218]:
# q_list와 doc_list 다시 나눈다.
doc_list = q_list[int(len(q_list) / 2):]
q_list = q_list[:int(len(q_list) / 2)]

In [219]:
# 질문 리스트 fit
tfidf_q_list = tfidf.transform(q_list).toarray()
# 문서 리스트 fit
tfidf_doc_list = tfidf.transform(doc_list).toarray()
tfidf_q_list.shape, tfidf_doc_list.shape

((960, 82763), (960, 82763))

In [220]:
# 코사인 유사도로 가장 높은 인덱스 추출
sim_matrix = cosine_similarity(tfidf_q_list, tfidf_doc_list)
sim_matrix.shape

(960, 960)

In [221]:
#idf 질의에 대한 적합한 문서 예측
y_hat = np.zeros((sim_matrix.shape[0]))
for i, q_ in enumerate(sim_matrix):
    y_hat[i] = (np.argmax(q_))
#예측한 문서인덱스를 데이터프레임에 적재
dataset['tfidf_y_hat'] = y_hat.astype(int)

In [222]:
dataset

,question,context,y,bm25_y_hat,tfidf_y_hat
0,일본의 독도 영유권 주장 당시 김영삼이 당대 외교활동을 강력하게 비꼬아 표현한 상대...,"""나는 대통령 때 외무부에 지시해 독도 인근 해역에 배를 엄청나게 띄워 해상시위를 ...",0,203,203
1,정부의 헌법개정안 준비 과정에 대해서 청와대 비서실이 아니라 국무회의 중심으로 이뤄...,"""내각과 장관들이 소외되고 대통령비서실의 권한이 너무 크다"", ""행보가 비서 본연의...",1,1,1
2,바이오쇼크에서 플레이어가 조심해야 하는 캐릭터는 무엇인가?,"""빅 대디""는 플레이어가 주의해야 하는 캐릭터로, 장갑한 잠수복을 입었으며 유전적으...",2,317,317
3,서양 철학이라는 용어의 지정학적 경계는 몇세기에 걸쳐 형성 되었는가?,"""서양 철학""이라는 용어의 지정학적 경계는 19-20세기에 걸쳐 형성되었다. 이 시...",3,3,3
4,고위급 탈북자는 천안함 침몰 사건이 북한이 몇년동안 철저히 준비해온 사건이라고 주장...,'북한의 군사문제와 군수산업에 정통한 한 고위급 탈북자'는 이 사건이 10년 동안 ...,4,4,4
...,...,...,...,...,...
955,1920년대 전후반에 일본은 무엇의 여파로 불황 상태에 빠졌나?,"히로히토가 천황 자리에 오르던 1920년대 전후반, 일본은 세계 대공황의 여파로 극...",955,955,955
956,가와무라 사후에 히로히토 형제의 양육을 맡았던 사람은?,히로히토는 가와무라가 세상을 떠난 1904년 11월까지 가와무라의 저택에서 지냈으며...,956,956,500
957,히로히토에게 박물과 물리를 가르쳤던 교수는?,"히로히토는 어렸을 때부터 자연에 관심을 보였다. 히로히토가 중등과에 다닐 때, 박물...",957,957,957
958,아몬의 일이 적혀있는 성경의 이름은 무엇인가?,"히브리어 성경에 기록된 바에 따르면, 아몬은 아버지 므낫세의 정책을 답습해 우상숭배...",958,958,958


In [223]:
print("tf-idf 정확도: ",round(dataset[dataset['y'] == dataset['tfidf_y_hat']].shape[0] /dataset.shape[0], 2))
print("bm25 정확도: ",round(dataset[dataset['y'] == dataset['bm25_y_hat']].shape[0] /dataset.shape[0], 2))

tf-idf 정확도:  0.71
bm25 정확도:  0.7


In [225]:
#TF-IDF 성능 검사
acc = accuracy_score(dataset['tfidf_y_hat'], dataset['y'])
print(acc)
cr = classification_report(dataset['y'], dataset['tfidf_y_hat'])
print(cr)

0.7083333333333334
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         1
           3       0.50      1.00      0.67         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           6       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         1
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       1.00      1.00      1.00         1
        

c:\Users\Lee\.conda\envs\mecab\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Lee\.conda\envs\mecab\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Lee\.conda\envs\mecab\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [201]:
#BM25 성능 검사
acc = accuracy_score(dataset['bm25_y_hat'], dataset['y'])
print(acc)
cr = classification_report(dataset['y'], dataset['bm25_y_hat'])
print(cr)

0.6916666666666667
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         1
           3       0.33      1.00      0.50         1
           4       1.00      1.00      1.00         1
           5       0.50      1.00      0.67         1
           6       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         1
           8       0.50      1.00      0.67         1
           9       0.50      1.00      0.67         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         1
          14       0.00      0.00      0.00         1
          15       1.00      1.00      1.00         1
          16       1.00      1.00      1.00         1
        

c:\Users\Lee\.conda\envs\mecab\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Lee\.conda\envs\mecab\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Lee\.conda\envs\mecab\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
